In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 
    'mid_padding', 
                 'strf_padding', 'ext_padding', 
                 'rnd_padding', 
    'aug_padding', 
    "zoom_padding"]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["zoom_padding"]
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons2/' #'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#'EC_number/archaea/rnn_conv/256rnn/'
#'EC_number/archaea/1conv/64filts_size5/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 2 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [314,77, 8]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task1/"
architecture = "only_denses"
#architecture = "conv_dense"
#architecture = "rnn_conv"
#architecture = "stack_conv"

In [3]:
lr= 0.0001 #0.001#
optimizer = Adam(lr=lr) #Adam(lr=lr)

### Loading stuff

In [4]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_split.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_split.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model  - Task 1

In [5]:
model = building_3dense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
#model = building_convrnn_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
#                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
#model = building_stackconv_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
#                                       kernel_size, pool_size, final_act, folder, optimizer)
generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task1', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 314)         8478      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 314)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 77)          24255     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 77)          0         
_________________________________________________________________
dense_3 (Dense)              (None, 1000, 8)           624       
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, val_generator,
                        architecture, max_lenn, dict_size, batch_size,
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val, 
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 314)         8478      
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 314)         0         
_________________________________________________________________
dense_6 (Dense)              (None, 1000, 77)          24255     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1000, 77)          0         
_________________________________________________________________
dense_7 (Dense)              (None, 1000, 8)           624       
__________

251/251 [==============================] - 6s 25ms/step - loss: 0.5974 - acc: 0.7041 - val_loss: 0.5765 - val_acc: 0.7324

Epoch 00021: val_acc improved from 0.73061 to 0.73235, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-021-0.7324.hdf5
Epoch 22/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5954 - acc: 0.7047 - val_loss: 0.5723 - val_acc: 0.7317

Epoch 00022: val_acc did not improve from 0.73235
Epoch 23/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5908 - acc: 0.7087 - val_loss: 0.5667 - val_acc: 0.7383

Epoch 00023: val_acc improved from 0.73235 to 0.73829, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-023-0.7383.hdf5
Epoch 24/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5897 - acc: 0.7081 - val_loss: 0.5630 - val_acc: 0.7372

Epo

251/251 [==============================] - 6s 25ms/step - loss: 0.5356 - acc: 0.7424 - val_loss: 0.4562 - val_acc: 0.8078

Epoch 00046: val_acc improved from 0.80748 to 0.80783, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-046-0.8078.hdf5
Epoch 47/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5378 - acc: 0.7410 - val_loss: 0.4551 - val_acc: 0.8082

Epoch 00047: val_acc improved from 0.80783 to 0.80818, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-047-0.8082.hdf5
Epoch 48/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5308 - acc: 0.7407 - val_loss: 0.4515 - val_acc: 0.8106

Epoch 00048: val_acc improved from 0.80818 to 0.81062, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improveme

251/251 [==============================] - 6s 25ms/step - loss: 0.5065 - acc: 0.7567 - val_loss: 0.4033 - val_acc: 0.8553

Epoch 00071: val_acc improved from 0.85360 to 0.85535, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-071-0.8553.hdf5
Epoch 72/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5056 - acc: 0.7606 - val_loss: 0.4028 - val_acc: 0.8564

Epoch 00072: val_acc improved from 0.85535 to 0.85639, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improvement-072-0.8564.hdf5
Epoch 73/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5075 - acc: 0.7580 - val_loss: 0.4010 - val_acc: 0.8578

Epoch 00073: val_acc improved from 0.85639 to 0.85779, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/post_padding/0/weights-improveme

251/251 [==============================] - 6s 25ms/step - loss: 0.4939 - acc: 0.7679 - val_loss: 0.3903 - val_acc: 0.8651

Epoch 00104: val_acc did not improve from 0.87142
Epoch 105/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4943 - acc: 0.7656 - val_loss: 0.3903 - val_acc: 0.8651

Epoch 00105: val_acc did not improve from 0.87142
Epoch 106/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4944 - acc: 0.7649 - val_loss: 0.3888 - val_acc: 0.8697

Epoch 00106: val_acc did not improve from 0.87142
Epoch 107/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4913 - acc: 0.7674 - val_loss: 0.3870 - val_acc: 0.8704

Epoch 00107: val_acc did not improve from 0.87142
Epoch 108/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4924 - acc: 0.7638 - val_loss: 0.3873 - val_acc: 0.8714

Epoch 00108: val_acc did not improve from 0.87142
Epoch 109/200
251/251 [==============================] - 6s 25ms/step - loss: 0

...

251/251 [==============================] - 6s 25ms/step - loss: 0.4987 - acc: 0.7632 - val_loss: 0.4890 - val_acc: 0.8183

Epoch 00116: val_acc did not improve from 0.82809
Epoch 117/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4969 - acc: 0.7635 - val_loss: 0.4862 - val_acc: 0.8208

Epoch 00117: val_acc did not improve from 0.82809
Epoch 118/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4971 - acc: 0.7644 - val_loss: 0.4871 - val_acc: 0.8194

Epoch 00118: val_acc did not improve from 0.82809
Epoch 119/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5001 - acc: 0.7632 - val_loss: 0.4907 - val_acc: 0.8173

Epoch 00119: val_acc did not improve from 0.82809
Epoch 120/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4935 - acc: 0.7655 - val_loss: 0.4866 - val_acc: 0.8204

Epoch 00120: val_acc did not improve from 0.82809
Epoch 121/200
251/251 [==============================] - 6s 25ms/step - loss: 0

251/251 [==============================] - 6s 25ms/step - loss: 0.4904 - acc: 0.7684 - val_loss: 0.4782 - val_acc: 0.8256

Epoch 00158: val_acc did not improve from 0.83124
Epoch 159/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4926 - acc: 0.7660 - val_loss: 0.4782 - val_acc: 0.8235

Epoch 00159: val_acc did not improve from 0.83124
Epoch 160/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4891 - acc: 0.7686 - val_loss: 0.4764 - val_acc: 0.8263

Epoch 00160: val_acc did not improve from 0.83124
Epoch 161/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4908 - acc: 0.7670 - val_loss: 0.4735 - val_acc: 0.8337

Epoch 00161: val_acc improved from 0.83124 to 0.83368, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/pre_padding/2/weights-improvement-161-0.8337.hdf5
Epoch 162/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4905 - acc: 0.7696 - val_loss

251/251 [==============================] - 6s 25ms/step - loss: 0.4821 - acc: 0.7741 - val_loss: 0.4528 - val_acc: 0.8505

Epoch 00195: val_acc improved from 0.85045 to 0.85045, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/pre_padding/2/weights-improvement-195-0.8505.hdf5
Epoch 196/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4859 - acc: 0.7722 - val_loss: 0.4547 - val_acc: 0.8452

Epoch 00196: val_acc did not improve from 0.85045
Epoch 197/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4847 - acc: 0.7717 - val_loss: 0.4562 - val_acc: 0.8456

Epoch 00197: val_acc did not improve from 0.85045
Epoch 198/200
251/251 [==============================] - 6s 25ms/step - loss: 0.4939 - acc: 0.7640 - val_loss: 0.4535 - val_acc: 0.8515

Epoch 00198: val_acc improved from 0.85045 to 0.85150, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/pr

251/251 [==============================] - 6s 25ms/step - loss: 0.5773 - acc: 0.7208 - val_loss: 0.5469 - val_acc: 0.7477

Epoch 00025: val_acc did not improve from 0.79734
Epoch 26/200
251/251 [==============================] - 6s 26ms/step - loss: 0.5736 - acc: 0.7202 - val_loss: 0.5423 - val_acc: 0.7516

Epoch 00026: val_acc did not improve from 0.79734
Epoch 27/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5710 - acc: 0.7224 - val_loss: 0.5391 - val_acc: 0.7523

Epoch 00027: val_acc did not improve from 0.79734
Epoch 28/200
251/251 [==============================] - 6s 26ms/step - loss: 0.5678 - acc: 0.7261 - val_loss: 0.5358 - val_acc: 0.7554

Epoch 00028: val_acc did not improve from 0.79734
Epoch 29/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5691 - acc: 0.7236 - val_loss: 0.5310 - val_acc: 0.7589

Epoch 00029: val_acc did not improve from 0.79734
Epoch 30/200
251/251 [==============================] - 6s 26ms/step - loss: 0.5654

251/251 [==============================] - 6s 26ms/step - loss: 0.5299 - acc: 0.7438 - val_loss: 0.4701 - val_acc: 0.8169

Epoch 00063: val_acc did not improve from 0.81971
Epoch 64/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5292 - acc: 0.7468 - val_loss: 0.4646 - val_acc: 0.8204

Epoch 00064: val_acc improved from 0.81971 to 0.82041, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/pre_padding/3/weights-improvement-064-0.8204.hdf5
Epoch 65/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5264 - acc: 0.7476 - val_loss: 0.4616 - val_acc: 0.8235

Epoch 00065: val_acc improved from 0.82041 to 0.82355, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task1/pre_padding/3/weights-improvement-065-0.8235.hdf5
Epoch 66/200
251/251 [==============================] - 6s 25ms/step - loss: 0.5267 - acc: 0.7482 - val_loss: 0.4631 - val_acc: 0.8242

Epoch